In [1]:
import numpy as np # linear algebra
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
os.environ["KERAS_BACKEND"] = "torch" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/math-qsa-dataset/train.csv
/kaggle/input/math-qsa-dataset/test.csv
/kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-prize/AIMO Prize - Note on Language and Notation.pdf
/kaggle/input/ai-mathematical-olympiad-prize/train.csv
/kaggle/input/ai-mathematical-olympiad-prize/test.csv
/kaggle/input/ai-mathematical-olympiad-prize/aimo/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/ai-mathematical-olympiad-prize/aimo/__init__.py
/kaggle/input/keras-lib-dataset/keras-3.3.3-py3-none-any.whl
/kaggle/input/keras-lib-dataset/keras_nlp-0.12.1-py3-none-any.whl
/kaggle/input/llama-3/transformers/8b-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-

2024-06-14 05:58:22.923090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 05:58:22.923196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 05:58:23.068441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install pylatexenc

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=4420e86087632caf8271ea4cc8167c0540518d5439cd60804db09ced9d516524
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [3]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/ai-mathematical-olympiad-prize"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train
keras.utils.set_random_seed(CFG.seed)

In [4]:
df1 = pd.read_csv("/kaggle/input/math-qsa-dataset/train.csv")
df2 = pd.read_csv("/kaggle/input/math-qsa-dataset/test.csv")
df = pd.concat([df1, df2], axis=0)

In [5]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    
df["is_integer"] = df.answer.map(is_integer)
df = df[df.is_integer].reset_index(drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   problem     7356 non-null   object
 1   level       7356 non-null   object
 2   type        7356 non-null   object
 3   solution    7356 non-null   object
 4   answer      7356 non-null   object
 5   is_integer  7356 non-null   bool  
dtypes: bool(1), object(5)
memory usage: 294.7+ KB


# Preprocessing Pipeline

In [7]:
import re
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Ensure the necessary NLTK data files are downloaded
nltk.download('punkt')
nltk.download('stopwords')

class Preprocessing:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))

    def convert_draw_command(self, draw_command):
        pattern_pentagon = re.compile(r'draw\(\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--cycle.*?\);')
        match_pentagon = pattern_pentagon.match(draw_command)
        if match_pentagon:
            coords = match_pentagon.groups()
            return f"A regular pentagon with vertices at {coords[0]}, {coords[1]}, {coords[2]}, {coords[3]}, and {coords[4]}."
        
        pattern_hexagon = re.compile(r'draw\(\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\)--\((.*?)\),.*?\);')
        match_hexagon = pattern_hexagon.match(draw_command)
        if match_hexagon:
            coords = match_hexagon.groups()
            return f"A regular hexagon with vertices at {coords[0]}, {coords[1]}, {coords[2]}, {coords[3]}, and {coords[4]}."
        
        return ""

    def convert_dot_label_commands(self, text):
        pattern_dot = re.compile(r'dot\(\((.*?)\)\);')
        text = pattern_dot.sub(r'A point at \1.', text)
        
        pattern_label = re.compile(r'label\("(.*?)",\((.*?)\),.*?\);')
        text = pattern_label.sub(r'The point \1 is at coordinates \2.', text)
        
        return text

    def preprocess_text(self, text):
        # Remove the [asy] tags
        text = re.sub(r'\[asy\]', '', text)
        text = re.sub(r'\[\/asy\]', '', text)

        # Split the text into commands
        commands = text.split('\n')

        readable_text = []
        for command in commands:
            if 'draw' in command:
                readable_text.append(self.convert_draw_command(command))
            else:
                readable_text.append(self.convert_dot_label_commands(command))

        readable_text = ' '.join(readable_text)

        # Tokenize into sentences
        sentences = sent_tokenize(readable_text)

        # Remove stop words and tokenize the remaining words
        filtered_sentences = []
        for sentence in sentences:
            word_tokens = word_tokenize(sentence)
            filtered_sentence = [word for word in word_tokens if word.lower() not in self.stop_words]
            filtered_sentences.append(' '.join(filtered_sentence))

        filtered_text = ' '.join(filtered_sentences)
        return filtered_text

    def process_dataframe(self, df, text_column):
        df[f'{text_column}'] = df[text_column].apply(self.preprocess_text)
        return df



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# pipeline = Preprocessing()
# df = pipeline.process_dataframe(df, "problem")

In [9]:
df

,problem,level,type,solution,answer,is_integer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3,True
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3,True
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28,True
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18,True
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132,True
...,...,...,...,...,...,...
7351,Find the number of real roots of\n\[2x^{2001} ...,Level 4,Intermediate Algebra,We can factor the given equation as\n\[(2x + 3...,1,True
7352,"For a complex number $z,$ compute the minimum ...",Level 4,Intermediate Algebra,"Geometrically, $|z + 5 - 3i|$ is the distance ...",13,True
7353,Compute the smallest positive integer $x$ grea...,Level 4,Intermediate Algebra,Let $q$ and $r$ be the remainder when $x$ is d...,1700,True
7354,"For positive real numbers $a,$ $b,$ $c,$ and $...",Level 5,Intermediate Algebra,"Let $S$ denote the given sum. First, we apply...",9,True


In [10]:
import sympy as sp
from sympy.parsing.latex import parse_latex
!pip install antlr4-python3-runtime
def latex_to_math(latex_str):
    """
    Convert a LaTeX string to a SymPy expression.

    Parameters:
    latex_str (str): The LaTeX string to convert.

    Returns:
    sympy.Expr: The corresponding SymPy expression.
    """
    try:
        # Parse the LaTeX string
        sympy_expr = parse_latex(latex_str)
        return sympy_expr
    except Exception as e:
        print(f"Error parsing LaTeX: {e}")
        return None

# Example usage
latex_str = r"\frac{d}{dx} \left( x^2 + 2x + 1 \right)"
math_expr = latex_to_math(latex_str)
print(math_expr)


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 4.5 MB/s eta 0:00:00
Error parsing LaTeX: LaTeX parsing requires the antlr4 Python package, provided by pip (antlr4-python3-runtime) or conda (antlr-python-runtime), version 4.11
None


# Prompt Engineering

In [11]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [12]:
df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = df.prompt.tolist()

  0%|          | 0/7356 [00:00<?, ?it/s]

In [13]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [14]:
# Take a random sample
sample = data[12]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
What is the largest positive multiple of $12$ that is less than $350?$



**<font color='cyan'>Solution:</font>**
Dividing $350$ by $12$ gives a quotient $29$ with a remainder of $2$. In other words, \[350=12\cdot29+2.\]Thus, $29\cdot12=\boxed{348}$ is the largest multiple of $12$ which is less than $350.$



**<font color='green'>Answer:</font>**
348

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   problem     7356 non-null   object
 1   level       7356 non-null   object
 2   type        7356 non-null   object
 3   solution    7356 non-null   object
 4   answer      7356 non-null   object
 5   is_integer  7356 non-null   bool  
 6   prompt      7356 non-null   object
dtypes: bool(1), object(6)
memory usage: 352.1+ KB


In [16]:
Markdown(colorize_text(df["prompt"][0]))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
The United States Postal Service charges an extra $\$0.11$ in postage if the length of an envelope, in inches, divided by its height, in inches, is less than $1.3$ or greater than $2.5.$ For how many of these four envelopes must the extra $\$0.11$ in postage be paid? \begin{tabular}[t]{ccc}
Envelope & Length in inches & Height in inches\\\hline
A &6 &4\\
B &9 &3\\
C &6 &6\\
D &11 &4
\end{tabular}



**<font color='cyan'>Solution:</font>**
We calculate the desired ratio for each envelope: \begin{align*}
\text{A} &= \frac{6}{4} = 1.5 \\
\text{B} &= \frac{9}{3} = 3 \\
\text{C} &= \frac{6}{6} = 1 \\
\text{D} &= \frac{11}{4} = 2.75
\end{align*} $\text B,$ $\text C,$ and $\text D$ are out of range, so the answer is $\boxed{3}.$



**<font color='green'>Answer:</font>**
3

In [17]:
df["solution"][0]

'We calculate the desired ratio for each envelope: \\begin{align*}\n\\text{A} &= \\frac{6}{4} = 1.5 \\\\\n\\text{B} &= \\frac{9}{3} = 3 \\\\\n\\text{C} &= \\frac{6}{6} = 1 \\\\\n\\text{D} &= \\frac{11}{4} = 2.75\n\\end{align*} $\\text B,$ $\\text C,$ and $\\text D$ are out of range, so the answer is $\\boxed{3}.$'

## Model Training

In [43]:
from transformers import AutoTokenizer
import transformers
import torch

model = "/kaggle/input/llama-3/transformers/8b-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model, model_max_length=512)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
def split_text_into_prompt_completion(df, text_column):
    prompts = []
    completions = []

    for index, row in df.iterrows():
        text = row[text_column]
        
        # Split based on "Solution:"
        problem_part, solution_part = text.split("Solution:", 1)
        
        # Ensure to keep the "Solution:" keyword in the completion
        solution_part = "Solution:" + solution_part
        
        # Append to lists
        prompts.append(problem_part.strip())
        completions.append(solution_part.strip())
    
    # Create new DataFrame
    split_df = pd.DataFrame({
        "prompt": prompts,
        "completion": completions
    })
    
    return split_df

# Example usage
data = split_text_into_prompt_completion(df, 'prompt')

In [45]:
data

,prompt,completion
0,Role:\nYou are an advanced AI system with exce...,Solution:\nWe calculate the desired ratio for ...
1,Role:\nYou are an advanced AI system with exce...,"Solution:\nA number with 15, 20 and 25 as fact..."
2,Role:\nYou are an advanced AI system with exce...,"Solution:\nDividing by $4$, we have $0<n<7\fra..."
3,Role:\nYou are an advanced AI system with exce...,Solution:\nWe will break up the problem into c...
4,Role:\nYou are an advanced AI system with exce...,Solution:\nWe know that the sum of the degree ...
...,...,...
7351,Role:\nYou are an advanced AI system with exce...,Solution:\nWe can factor the given equation as...
7352,Role:\nYou are an advanced AI system with exce...,"Solution:\nGeometrically, $|z + 5 - 3i|$ is th..."
7353,Role:\nYou are an advanced AI system with exce...,Solution:\nLet $q$ and $r$ be the remainder wh...
7354,Role:\nYou are an advanced AI system with exce...,Solution:\nLet $S$ denote the given sum. Firs...


In [46]:
tokenized_data = []
prompt = data["prompt"]
completion = data["completion"]
input_texts = [prompt + tokenizer.eos_token + completion for prompt, completion in zip(data["prompt"], data["completion"])]

for input_text in input_texts:
    tokenized_input = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
    tokenized_data.append(tokenized_input)

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
from datasets import Dataset

# Create a list of dictionaries with 'input_ids' and 'attention_mask'
train_data = []
for item in tokenized_data:
    train_data.append({
        'input_ids': item['input_ids'].squeeze(),
        'attention_mask': item['attention_mask'].squeeze()
    })

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_dict(train_data)

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

model = AutoModelForCausalLM.from_pretrained(model, torch_dtype=torch.float16)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=200,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

In [ ]:
trainer.save_model("path_to_save_your_model")
tokenizer.save_pretrained("path_to_save_your_model")

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained("path_to_save_your_model")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("path_to_save_your_model")

# Create a pipeline
generation_pipeline = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=fine_tuned_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Generate text
prompt = "Your test prompt"
generated_text = generation_pipeline(prompt)
print(generated_text)